# Example of ensemble runs


In [3]:
import numpy as np
import pandas as pd
import yaml
import h5py
from tqdm.notebook import tqdm
from mce.core.driver import Driver
from mce.util.io import retrieve_url

## Parameter ensemble

In [4]:
# Definition of parameters
with open('mce/core/attributes.yml') as f1:
    atts = yaml.safe_load(f1)['parameters']

In [5]:
def get_parms_ens(f_parms, cat, names, members):
    """Read parameter ensemble

    Parameters
    ----------
    f_parms
        h5py.File object
    cat
        Parameter category
    names
        Parameter names
    members
        Members

    Returns
    -------
        Array of selected parameters
    """
    return np.vstack([
        f_parms[f'{cat}/{name}'][members]
        for name in names
    ]).T

# Parameter names by category
pn_climate = ['a0', 'a1', 'a2', 'tau0', 'tau1', 'tau2', 'lambda']
pn_forcing = ['co2_alpha', 'co2_beta']
pn_forcing_scale = [
    'ch4', 'n2o', 'f_gases', 'montreal_gases',
    'o3_strat', 'o3_trop',
    'aerosols', 'bc_on_snow', 'land_use', 'h2o_strat',
]
pn_land = ['beta', 'fb_alpha']
pn_ocean = ['hls', 'hl1', 'hl2', 'hl3', 'eta1', 'eta2', 'eta3']

In [21]:
# Fetch two sets of parameter ensembles,
# a CMIP-based ensemble as a prior and a constrained ensemble from the prior
# Data files are not publicly available

labels_ens = ['Prior', 'Constrained']

# Use arbitrary 600 members
mm_prior = pd.read_csv(
    'untracked/parms_perturbed_members_prior.csv', index_col=0
).squeeze().tolist()[1000:1000+600]

mm_constrained = pd.read_csv(
    'untracked/parms_perturbed_members_constrained.csv',
    index_col=0
).squeeze().tolist()[-600:]

with h5py.File('untracked/parms_unconstrained_rcmip2.h5', 'r') as f1:
    # file contents
    f1.visititems(
        lambda name, obj:
        print(name, '(group)') if isinstance(obj, h5py.Group)
        else print(
            '{}: {}'.format(
                name,
                atts[name.split('/')[0]][name.split('/')[-1]]['long_name'],
            )
        )
    )
    # ensemble size
    print('Total unconstrained', f1['climate/a0'].shape[0])

    parms_ens = {
        label:
        {
            'climate': get_parms_ens(f1, 'climate', pn_climate, mm),
            'forcing': get_parms_ens(f1, 'forcing', pn_forcing, mm),
            'forcing_scale':
            get_parms_ens(f1, 'forcing_scale', pn_forcing_scale, mm),
            'ocean_1750': get_parms_ens(f1, 'ocean/init_1750', pn_ocean, mm),
            'ocean_1850': get_parms_ens(f1, 'ocean/init_1850', pn_ocean, mm),
            'land': get_parms_ens(f1, 'land', pn_land, mm),
        }
        for label, mm in
        zip(labels_ens, [mm_prior, mm_constrained])
    }

for k, v in parms_ens.items():
    print(k, v['climate'].shape[0])

climate (group)
climate/a0: Fraction of the first time constant contribution to the surface temperature change
climate/a1: Fraction of the second time constant contribution to the surface temperature change
climate/a2: Fraction of the third time constant contribution to the surface temperature change
climate/ecs: Equilibrium climate sensitivity
climate/ecs_reg: Equilibrium climate sensitivity based on conventional linear regression
climate/lambda: Climate feedback parameter
climate/tau0: First (shortest) time constant
climate/tau1: Second time constant
climate/tau2: Third time constant
climate/tcr: Transient climate response
forcing (group)
forcing/co2_alpha: Scaling factor of CO2 forcing
forcing/co2_beta: Amplification factor of CO2 forcing
forcing_scale (group)
forcing_scale/aerosols: Scaling factor for aerosols
forcing_scale/bc_on_snow: Scaling factor for surface albedo due to light-absorbing particles on snow and ice
forcing_scale/ch4: Scaling factor for CH4
forcing_scale/f_gases: 

In [22]:
def get_parms_driver(parms, year_pi, cco2_pi, ccref=None):
    """Return keyword arguments for driver

    Parameters
    ----------
    parms
        Categorized parameters
    cco2_pi
        Pre-industrial base year for the ocean module
    cco2_pi
        Pre-industrial CO2 concentration for the ocean module
    ccref, optional
        Reference CO2 concentration of CO2 forcing
        If None (default), use cco2_pi

    Returns
    -------
        Keyword arguments
    """
    if ccref is None:
        ccref = cco2_pi

    return {
        'kw_irm': {
            'asj': parms['climate'][0:3],
            'tauj': parms['climate'][3:6],
            'lamb': parms['climate'][6],
        },
        'kw_rfall': {
            'ccref': ccref,
            'alpha': parms['forcing'][0],
            'beta': parms['forcing'][1],
        },
        'kw_ocean': {
            'cco2_pi': cco2_pi,
            'hlk': parms[f'ocean_{year_pi}'][0:4],
            'etak': parms[f'ocean_{year_pi}'][4:7],
        },
        'kw_land': {
            'beta': parms['land'][0],
            'fb_alpha': parms['land'][1],
        },
    }

## Scenario data from RCMIP2

In [8]:
url = 'https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0'

fn = 'rcmip-emissions-annual-means-v5-1-0.csv'
df = pd.read_csv(retrieve_url(f'./datain/rcmip/{fn}', f'{url}/{fn}'))
df_rcmip_emis = (
    df
    .loc[df['Region']=='World']
    .set_index(df.columns[:7].to_list())
    .rename(columns=int)
)

fn = 'rcmip-concentrations-annual-means-v5-1-0.csv'
df = pd.read_csv(retrieve_url(f'./datain/rcmip/{fn}', f'{url}/{fn}'))
df_rcmip_conc = (
    df
    .loc[df['Region']=='World']
    .set_index(df.columns[:7].to_list())
    .rename(columns=int)
)

fn = 'rcmip-radiative-forcing-annual-means-v5-1-0.csv'
df = pd.read_csv(retrieve_url(f'./datain/rcmip/{fn}', f'{url}/{fn}'))
df_rcmip_rf = (
    df
    .set_index(df.columns[:7].to_list())
    .rename(columns=int)
)

[2024-10-05 17:51:45 mce.util.io] INFO:Use local file datain/rcmip/rcmip-emissions-annual-means-v5-1-0.csv retrieved from https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/rcmip-emissions-annual-means-v5-1-0.csv on 2024-05-25
[2024-10-05 17:51:46 mce.util.io] INFO:Use local file datain/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv retrieved from https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/rcmip-concentrations-annual-means-v5-1-0.csv on 2024-06-20
[2024-10-05 17:51:46 mce.util.io] INFO:Use local file datain/rcmip/rcmip-radiative-forcing-annual-means-v5-1-0.csv retrieved from https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/rcmip-radiative-forcing-annual-means-v5-1-0.csv on 2024-09-28


In [9]:
class InputRcmip:
    def __init__(self, df_emis, df_conc, df_rf):
        self.df_emis = df_emis
        self.df_conc = df_conc
        self.df_rf = df_rf

        self.id_vars = df_emis.index.names

    def _rename_species(self, x):
        return (
            x
            .split('|')[-1]
            .replace('CFC', 'CFC-')
            # .replace('HCFC', 'HCFC-')
            .replace('Halon', 'Halon-')
            .replace('HFC', 'HFC-')
            .replace('HFC-4310mee', 'HFC-43-10mee')
            .replace('C4F10', 'n-C4F10')
            .replace('C5F12', 'n-C5F12')
            .replace('C6F14', 'n-C6F14')
            .replace('cC4F8', 'c-C4F8')
        )

    def get_cco2(self, scenario, year):
        if isinstance(year, (tuple, list)):
            year = slice(*year)

        d_cco2 = (
            self.df_conc
            .groupby(['Scenario', 'Variable'])
            .get_group((scenario, 'Atmospheric Concentrations|CO2'))
            .squeeze()
            .dropna()
            .loc[year]
        )

        return d_cco2

    def get_cghg(self, scenario, year):
        if isinstance(year, (tuple, list)):
            year = slice(*year)

        id_vars_excl = [
            x for x in self.id_vars if x not in ['Variable', 'Unit']
        ]

        df_cghg = (
            self.df_conc
            .groupby('Scenario')
            .get_group(scenario)
            .dropna(axis=1)
            .droplevel(id_vars_excl)
            .rename(self._rename_species, level='Variable')
            .loc[:, year]
        )

        return df_cghg

    def get_rf_other(self, scenario, year):
        if isinstance(year, (tuple, list)):
            year = slice(*year)

        id_vars_excl = [
            x for x in self.id_vars if x not in ['Variable']
        ]
        df = (
            self.df_rf
            .groupby('Scenario')
            .get_group(scenario)
            .dropna(axis=1)
            .droplevel(id_vars_excl)
            .loc[:, year]
        )

        map_variable_rf = {
            'Aerosols': 'aerosols',
            'Tropospheric Ozone': 'o3_trop',
            'Stratospheric Ozone': 'o3_strat',
            'Albedo Change': 'land_use',
            'BC on Snow': 'bc_on_snow',
            'CH4 Oxidation Stratospheric H2O': 'h2o_strat',
            'Contrails and Contrail-induced Cirrus': 'contrails',
            'Solar': 'solar',
            'Volcanic': 'volcanic',
        }

        df_rf_other = df.loc[[
            x.split('|')[-1] in map_variable_rf
            for x in df.index.get_level_values('Variable')
        ]].rename(
            lambda x: map_variable_rf[x.split('|')[-1]],
            level='Variable',
        )

        return df_rf_other

input = InputRcmip(df_rcmip_emis, df_rcmip_conc, df_rcmip_rf)

In [10]:
cco2_1750 = input.get_cco2('historical', 1750)
cco2_1850 = input.get_cco2('historical', 1850)
cco2_1750, cco2_1850

(277.1470031738281, 284.3169987996419)

## CO2-only concentration-driven runs

In [25]:
# Using an idealized 1%-per-year increase trajectory
scenario = '1pctCO2'
d_cco2 = input.get_cco2(scenario, (1850, 2000))

In [26]:
# Define output variables
seconds_per_year = 3.15569e7
earth_area = 5.10e14 # m^2
outform = {
    'variables': [
        'cbst', 'eco2', 'coct', 'ctot', 'erf',
        'tas', 'thc', 'rtnt', 'abf', 'tcre',
    ],
    'conversion': [
        # from J/spy/m^2 (spy=seconds per year) to ZJ
        ('thc', seconds_per_year * earth_area * 1e-21, 'ZJ'),
    ],
}

In [27]:
nsize = 600
dfout_ens = {k: {} for k in parms_ens}

for name, parms in tqdm(parms_ens.items(), leave=False):
    for im in tqdm(range(nsize), leave=False):
        parms1 = {k: v[im] for k, v in parms.items()}
        drv = Driver(
            d_cco2.index.values, cco2=d_cco2.values,
            **get_parms_driver(parms1, 1850, cco2_1850),
        )
        df = drv.run(outform)
        dfout_ens[name][im] = df

    dfout_ens[name] = pd.concat(dfout_ens[name], names=['Member', 'Variable'])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [28]:
outpath = 'untracked/out_ens_rcmip2.h5'

with h5py.File(outpath, 'w') as f1:
    f1.create_dataset(f'{scenario}/input/time', data=d_cco2.index.values)
    f1.create_dataset(f'{scenario}/input/cco2', data=d_cco2.values)

    for ensemble, df in dfout_ens.items():
        for variable, df1 in df.groupby('Variable'):
            f1.create_dataset(
                f'{scenario}/{ensemble}/{variable}',
                data=df1.droplevel('Variable').values,
                chunks=True,
            )

# Concentration-driven historical runs

Using multi-agents forcing, staring from 1750

In [29]:
scenario = 'historical'
df_cghg = input.get_cghg(scenario, (1750, None))
df_rf_other = input.get_rf_other(scenario, (1750, None))

In [30]:
# GHG concentrations to forcing without climate interaction
drv = Driver([0], cco2=[cco2_1750]) # dummy driver instance
df = drv.forcing.ghg_concentrations_to_forcing(df_cghg).droplevel(1)
df_erf_ghg = df.groupby([drv.forcing.ghgs[x].category for x in df.index]).sum()

In [31]:
# Categorized non-CO2 forcing
df_rf_base = pd.concat([df_erf_ghg.drop('co2'), df_rf_other])
df_rf_base

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
ch4,0.001429,0.001709,0.002395,0.002870,0.003237,0.003581,0.004023,0.004328,0.004530,0.004461,...,0.513092,0.513079,0.515174,0.518764,0.520141,0.522299,0.524243,0.525038,0.527779,0.531101
f_gases,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.021934,0.023631,0.025376,0.027170,0.028682,0.030658,0.032837,0.034761,0.037583,0.039603
montreal_gases,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.361339,0.361356,0.361272,0.361443,0.361155,0.361117,0.360692,0.360061,0.359833,0.358995
n2o,0.013883,0.013989,0.014110,0.014278,0.014436,0.014564,0.014666,0.014729,0.014849,0.014982,...,0.167234,0.169841,0.172132,0.175101,0.177350,0.180116,0.183376,0.186088,0.188982,0.192377
aerosols,0.000000,0.000836,0.001212,0.000801,-0.000571,0.000063,0.000618,-0.001421,-0.000764,-0.000720,...,-1.387485,-1.398719,-1.388067,-1.367507,-1.329220,-1.321976,-1.330262,-1.320120,-1.308953,-1.308580
land_use,0.000000,-0.000229,-0.000459,-0.000688,-0.000918,-0.001147,-0.001377,-0.001606,-0.001836,-0.002065,...,-0.195649,-0.195983,-0.196318,-0.196653,-0.196988,-0.197322,-0.197657,-0.197992,-0.198326,-0.198661
bc_on_snow,0.000000,-0.000281,-0.000378,-0.000291,0.000017,0.000016,-0.000326,0.000566,0.000257,0.000220,...,0.076119,0.077084,0.081136,0.078962,0.079160,0.085433,0.083331,0.085887,0.084253,0.086328
h2o_strat,0.000000,0.000035,0.000121,0.000180,0.000226,0.000269,0.000325,0.000363,0.000388,0.000379,...,0.065072,0.065069,0.065338,0.065800,0.065977,0.066255,0.066504,0.066606,0.066959,0.067386
contrails,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020012,0.020250,0.020722,0.020471,0.019395,0.020269,0.020810,0.021098,0.021369,0.021474
o3_strat,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.057565,-0.056642,-0.056207,-0.055470,-0.054435,-0.053786,-0.053214,-0.052651,-0.052049,-0.051391


In [32]:
# Define output variables
outform = {
    'variables': [
        'cbst', 'eco2', 'coct', 'ctot', 'erf',
        'tas', 'thc', 'rtnt', 'abf', 'tcre',
        'erf|CO2', 'erf|non-CO2',
    ],
    'variables_derived': [
        # derive gmst and ohc from tas and thc by simple scaling
        ('gmst', ['tas', 1.04, '/'],
        'Surface air ocean blended temperature change', 'degC'),
        ('ohc', ['thc', 1.08, '/'],
        'Ocean heat content change', 'J/spy/m^2'), 
    ],
    'conversion': [
        # from J/spy/m^2 (spy=seconds per year) to ZJ
        ('thc', seconds_per_year * earth_area * 1e-21, 'ZJ'),
        ('ohc', seconds_per_year * earth_area * 1e-21, 'ZJ'),
    ],
}

In [33]:
d_cco2 = df_cghg.loc[('CO2', 'ppm')]
dfout_ens = {k: {} for k in parms_ens}

for name, parms in tqdm(parms_ens.items(), leave=False):
    for im in tqdm(range(nsize), leave=False):
        parms1 = {k: v[im] for k, v in parms.items()}
        d_scale = pd.Series(
            parms1['forcing_scale'],
            index=pn_forcing_scale,
        ).reindex(df_rf_base.index, fill_value=1.)
        df_erf_nonco2 = df_rf_base.mul(d_scale, axis=0)
        drv = Driver(
            d_cco2.index.values, cco2=d_cco2.values,
            erf_nonco2=df_erf_nonco2.sum().values,
            **get_parms_driver(parms1, 1750, cco2_1750),
        )
        df = drv.run(outform)
        dfout_ens[name][im] = df

    dfout_ens[name] = pd.concat(dfout_ens[name], names=['Member', 'Variable'])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [34]:
with h5py.File(outpath, 'a') as f1:
    f1.create_dataset(f'{scenario}/input/time', data=d_cco2.index.values)
    f1.create_dataset(f'{scenario}/input/cco2', data=d_cco2.values)

    for k, v in df_rf_base.iterrows():
        f1.create_dataset(f'{scenario}/input/rf__{k}', data=v.values)

    for ensemble, df in dfout_ens.items():
        for variable, df1 in df.groupby('Variable'):
            variable = (
                variable
                .replace('|', '__')
                .replace('non-CO2', 'nonco2')
                .replace('CO2', 'co2')
            )
            f1.create_dataset(
                f'{scenario}/{ensemble}/{variable}',
                data=df1.droplevel('Variable').values,
                chunks=True,
            )

# Concentration-driven SSP scenario runs

Using multi-agents forcing of SSP2-4.5, staring from 1850

In [35]:
scenario = 'ssp245'
# Here, read data from 1750, but drop the period of 1750-1849
df_cghg = input.get_cghg(scenario, (1750, 2100))
df_rf_other = input.get_rf_other(scenario, (1750, 2100))

In [36]:
# GHG concentrations to forcing without climate interaction
drv = Driver([0], cco2=[cco2_1750]) # dummy driver instance
df = drv.forcing.ghg_concentrations_to_forcing(df_cghg).droplevel(1)
df_erf_ghg = df.groupby([drv.forcing.ghgs[x].category for x in df.index]).sum()

In [37]:
# Categorized non-CO2 forcing
# 1750-1849 period is dropped; no adjustment is applied
df_rf_base = pd.concat([df_erf_ghg.drop('co2'), df_rf_other]).loc[:, 1850:]
df_rf_base

,1850,1851,1852,1853,1854,1855,1856,1857,1858,1859,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
ch4,4.977546e-02,4.987614e-02,5.033398e-02,5.109210e-02,5.190273e-02,5.287833e-02,5.377010e-02,5.477777e-02,5.593447e-02,5.717304e-02,...,0.483927,0.482260,0.480673,0.479167,0.477731,0.476364,0.475060,0.473818,0.472630,0.471493
f_gases,3.017980e-08,3.288247e-08,3.593382e-08,3.899821e-08,4.207088e-08,4.515681e-08,4.825765e-08,5.137505e-08,5.451063e-08,5.776008e-08,...,0.155949,0.156489,0.156982,0.157431,0.157840,0.158209,0.158540,0.158836,0.159098,0.159328
montreal_gases,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.121738,0.120226,0.118735,0.117266,0.115820,0.114394,0.112990,0.111606,0.110243,0.108899
n2o,1.073941e-02,1.100692e-02,1.127779e-02,1.162905e-02,1.199117e-02,1.238240e-02,1.277721e-02,1.313148e-02,1.342692e-02,1.393447e-02,...,0.342685,0.343230,0.343735,0.344205,0.344639,0.345036,0.345397,0.345725,0.346016,0.346272
aerosols,-3.947552e-02,-3.826972e-02,-3.870359e-02,-4.055589e-02,-4.847449e-02,-4.791003e-02,-5.021529e-02,-5.218209e-02,-5.370057e-02,-5.644665e-02,...,-0.527044,-0.523812,-0.520573,-0.517328,-0.514076,-0.510818,-0.507553,-0.504281,-0.501002,-0.497717
land_use,-3.125688e-02,-3.196075e-02,-3.266462e-02,-3.336848e-02,-3.407235e-02,-3.477622e-02,-3.548009e-02,-3.618396e-02,-3.688782e-02,-3.759169e-02,...,-0.193391,-0.192117,-0.190828,-0.189526,-0.188210,-0.186880,-0.185536,-0.184178,-0.182806,-0.181421
bc_on_snow,5.343449e-03,4.842183e-03,4.626548e-03,4.854602e-03,5.782621e-03,5.594082e-03,5.369415e-03,6.081800e-03,6.348239e-03,6.213890e-03,...,0.010035,0.009614,0.009193,0.008772,0.008351,0.007930,0.007509,0.007088,0.006667,0.006246
h2o_strat,6.061261e-03,6.073903e-03,6.131403e-03,6.226620e-03,6.328439e-03,6.450986e-03,6.563009e-03,6.689603e-03,6.834931e-03,6.990555e-03,...,0.061248,0.061034,0.060829,0.060635,0.060450,0.060275,0.060107,0.059947,0.059794,0.059648
contrails,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.356368e-10,2.322624e-10,4.785243e-10,7.666280e-10,7.860864e-10,...,0.077742,0.078649,0.079556,0.080462,0.081369,0.082276,0.083182,0.084089,0.084996,0.085902
o3_strat,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-0.012453,-0.012230,-0.012015,-0.011807,-0.011603,-0.011403,-0.011208,-0.011017,-0.010829,-0.010646


In [39]:
d_cco2 = df_cghg.loc[('CO2', 'ppm'), 1850:]
dfout_ens = {k: {} for k in parms_ens}

for name, parms in tqdm(parms_ens.items(), leave=False):
    for im in tqdm(range(nsize), leave=False):
        parms1 = {k: v[im] for k, v in parms.items()}
        d_scale = pd.Series(
            parms1['forcing_scale'],
            index=pn_forcing_scale,
        ).reindex(df_rf_base.index, fill_value=1.)
        df_erf_nonco2 = df_rf_base.mul(d_scale, axis=0)
        # forcing base year is set to 1750
        drv = Driver(
            d_cco2.index.values, cco2=d_cco2.values,
            erf_nonco2=df_erf_nonco2.sum().values,
            **get_parms_driver(parms1, 1850, cco2_1850, cco2_1750),
        )
        df = drv.run(outform)
        dfout_ens[name][im] = df

    dfout_ens[name] = pd.concat(dfout_ens[name], names=['Member', 'Variable'])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [40]:
with h5py.File(outpath, 'a') as f1:
    f1.create_dataset(f'{scenario}/input/time', data=d_cco2.index.values)
    f1.create_dataset(f'{scenario}/input/cco2', data=d_cco2.values)

    for k, v in df_rf_base.iterrows():
        f1.create_dataset(f'{scenario}/input/rf__{k}', data=v.values)

    for ensemble, df in dfout_ens.items():
        for variable, df1 in df.groupby('Variable'):
            variable = (
                variable
                .replace('|', '__')
                .replace('non-CO2', 'nonco2')
                .replace('CO2', 'co2')
            )
            f1.create_dataset(
                f'{scenario}/{ensemble}/{variable}',
                data=df1.droplevel('Variable').values,
                chunks=True,
            )